In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import Levenshtein
import numpy as np
import math
import statistics
from sklearn.metrics import confusion_matrix

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve
import numpy as np
from tqdm import tqdm

In [3]:
dataset1 = pd.read_csv("dataset1.csv")
dataset1

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,girl_median,girl_pstdev,girl_max,girl_min,gihl_count,gihl_mean,gihl_median,gihl_pstdev,gihl_max,gihl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,160159_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27353,349891_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27354,144723_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27355,163905_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dataset2 = pd.read_csv("dataset2.csv")
dataset2

,User ID #1,User ID #2,Label
0,21833_1,21833_2,0
1,233209_2,233209_1,0
2,39734_2,39734_1,0
3,285679_2,285679_1,0
4,206052_1,206052_2,0
...,...,...,...
25701,183297_1,154453_2,1
25702,44951_2,342622_1,1
25703,388515_1,62296_2,1
25704,11462_1,82765_2,1


In [5]:
import numpy as np
from tqdm import tqdm

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1)
X, y = process_dataset(dataset2, user_features_gen)

print(X)
print(y)

Processing rows: 100%|██████████| 25706/25706 [00:31<00:00, 808.25it/s]


[[0.00000000e+00 1.62817600e+08 1.62817600e+08 ...            nan
             nan            nan]
 [           nan            nan            nan ...            nan
             nan            nan]
 [           nan            nan            nan ...            nan
             nan            nan]
 ...
 [           nan            nan            nan ...            nan
             nan            nan]
 [           nan            nan            nan ...            nan
             nan            nan]
 [4.00000000e+00 7.38310562e+07 7.07869822e+07 ...            nan
             nan            nan]]
[0 0 0 ... 1 1 1]


In [6]:
X

array([[0.00000000e+00, 1.62817600e+08, 1.62817600e+08, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       ...,
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [4.00000000e+00, 7.38310562e+07, 7.07869822e+07, ...,
                   nan,            nan,            nan]])

In [7]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [8]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Accuracy: 0.7354497354497355
Confusion Matrix:
[[1077  520]
 [ 330 1286]]
FPR for Test Set: 0.2542266750156543
TPR for Test Set: 0.7456683168316832
EER for Test Set: 0.25427917909198555
Validation Accuracy: 0.7252644679527069
Validation Confusion Matrix:
[[1107  503]
 [ 380 1224]]
FPR for Validation Set: 0.26211180124223604
TPR for Validation Set: 0.7381546134663342
EER for Validation Set: 0.261978593887951


In [9]:
dataset1a = pd.read_csv("dataset1.csv")
dataset1a

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,girl_median,girl_pstdev,girl_max,girl_min,gihl_count,gihl_mean,gihl_median,gihl_pstdev,gihl_max,gihl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,160159_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27353,349891_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27354,144723_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27355,163905_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
dataset1a.fillna(0, inplace=True)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1a)
X, y = process_dataset(dataset2, user_features_gen)

Processing rows: 100%|██████████| 25706/25706 [00:32<00:00, 798.72it/s]


In [11]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Accuracy: 0.6591970121381886
Confusion Matrix:
[[1136  461]
 [ 634  982]]
FPR for Test Set: 0.36318096430807767
TPR for Test Set: 0.6373762376237624
EER for Test Set: 0.36290236334215764
Validation Accuracy: 0.6505911636589919
Validation Confusion Matrix:
[[1157  453]
 [ 670  934]]
FPR for Validation Set: 0.36770186335403726
TPR for Validation Set: 0.6321695760598504
EER for Validation Set: 0.3677661436470934


In [12]:
dataset1b = pd.read_csv("dataset1.csv")
dataset1b

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,girl_median,girl_pstdev,girl_max,girl_min,gihl_count,gihl_mean,gihl_median,gihl_pstdev,gihl_max,gihl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,160159_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27353,349891_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27354,144723_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27355,163905_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
median_values = dataset1b.median()
dataset1b.fillna(median_values, inplace=True)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1b)
X, y = process_dataset(dataset2, user_features_gen)

Processing rows: 100%|██████████| 25706/25706 [00:26<00:00, 984.57it/s] 


In [14]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Accuracy: 0.6181139122315593
Confusion Matrix:
[[1025  572]
 [ 655  961]]
FPR for Test Set: 0.3882279273638071
TPR for Test Set: 0.6120049504950495
EER for Test Set: 0.3881114884343788
Validation Accuracy: 0.6225886745488488
Validation Confusion Matrix:
[[1055  555]
 [ 658  946]]
FPR for Validation Set: 0.3826086956521739
TPR for Validation Set: 0.6172069825436409
EER for Validation Set: 0.3827008565542665


In [15]:
dataset1c = pd.read_csv("dataset1.csv")
dataset1c

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,girl_median,girl_pstdev,girl_max,girl_min,gihl_count,gihl_mean,gihl_median,gihl_pstdev,gihl_max,gihl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,160159_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27353,349891_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27354,144723_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27355,163905_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
mean_values = dataset1c.mean()
dataset1c.fillna(mean_values, inplace = True)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1c)
X, y = process_dataset(dataset2, user_features_gen)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Processing rows: 100%|██████████| 25706/25706 [00:26<00:00, 959.77it/s] 


Accuracy: 0.6707127295362589
Confusion Matrix:
[[1159  438]
 [ 620  996]]
FPR for Test Set: 0.3544145272385723
TPR for Test Set: 0.6460396039603961
EER for Test Set: 0.35418746163908815
Validation Accuracy: 0.6689483509645302
Validation Confusion Matrix:
[[1200  410]
 [ 654  950]]
FPR for Validation Set: 0.35093167701863354
TPR for Validation Set: 0.649002493765586
EER for Validation Set: 0.3509645916265237


In [17]:
dataset1d = pd.read_csv("dataset1.csv")
dataset1d

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,girl_median,girl_pstdev,girl_max,girl_min,gihl_count,gihl_mean,gihl_median,gihl_pstdev,gihl_max,gihl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,160159_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27353,349891_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27354,144723_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27355,163905_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
dataset1d = dataset1d.applymap(lambda x: np.log1p(x) if isinstance(x, (int, float)) else x)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1d)
X, y = process_dataset(dataset2, user_features_gen)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Processing rows: 100%|██████████| 25706/25706 [00:25<00:00, 1018.38it/s]


Accuracy: 0.7525676937441643
Confusion Matrix:
[[1117  480]
 [ 315 1301]]
FPR for Test Set: 0.23731997495303694
TPR for Test Set: 0.7629950495049505
EER for Test Set: 0.2371624627240433
Validation Accuracy: 0.7507778469197262
Validation Confusion Matrix:
[[1154  456]
 [ 345 1259]]
FPR for Validation Set: 0.24472049689440994
TPR for Validation Set: 0.7556109725685786
EER for Validation Set: 0.24455476216291566


In [19]:
dataset1e = pd.read_csv("dataset1.csv")
dataset1e

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,girl_median,girl_pstdev,girl_max,girl_min,gihl_count,gihl_mean,gihl_median,gihl_pstdev,gihl_max,gihl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,160159_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27353,349891_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27354,144723_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27355,163905_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
dataset1e = dataset1e.applymap(lambda x: np.log1p(x) if isinstance(x, (int, float)) else x)
mean_values = dataset1e.mean()
dataset1e.fillna(mean_values, inplace = True)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1e)
X, y = process_dataset(dataset2, user_features_gen)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Processing rows: 100%|██████████| 25706/25706 [00:25<00:00, 1024.69it/s]


Accuracy: 0.6738250855897915
Confusion Matrix:
[[1188  409]
 [ 639  977]]
FPR for Test Set: 0.3512836568566061
TPR for Test Set: 0.6497524752475248
EER for Test Set: 0.3507655908045406
Validation Accuracy: 0.6776602364654636
Validation Confusion Matrix:
[[1236  374]
 [ 662  942]]
FPR for Validation Set: 0.3515527950310559
TPR for Validation Set: 0.6502493765586035
EER for Validation Set: 0.35065170923622624


In [21]:
dataset1f = pd.read_csv("dataset1.csv")
dataset1f

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,girl_median,girl_pstdev,girl_max,girl_min,gihl_count,gihl_mean,gihl_median,gihl_pstdev,gihl_max,gihl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,160159_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27353,349891_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27354,144723_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27355,163905_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
dataset1f = dataset1f.applymap(lambda x: np.log1p(x) if isinstance(x, (int, float)) else x)
mean_values = dataset1f.mean()
dataset1f.fillna(mean_values, inplace = True)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1f)
X, y = process_dataset(dataset2, user_features_gen)

Processing rows: 100%|██████████| 25706/25706 [00:23<00:00, 1082.37it/s]


In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
combined2 = scaler.fit_transform(X)
columns = [f"Feature_{i}" for i in range(combined2.shape[1])]
combined2_df_s = pd.DataFrame(combined2, columns=columns)
X = combined2_df_s

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Accuracy: 0.6840958605664488
Confusion Matrix:
[[1219  378]
 [ 637  979]]
FPR for Test Set: 0.34815278647463993
TPR for Test Set: 0.6491336633663366
EER for Test Set: 0.34950956155415164
Validation Accuracy: 0.672682016179216
Validation Confusion Matrix:
[[1214  396]
 [ 656  948]]
FPR for Validation Set: 0.35838509316770184
TPR for Validation Set: 0.6415211970074813
EER for Validation Set: 0.3584319480801102


In [24]:
dataset1g = pd.read_csv("dataset1.csv")
dataset1g

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,girl_median,girl_pstdev,girl_max,girl_min,gihl_count,gihl_mean,gihl_median,gihl_pstdev,gihl_max,gihl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,160159_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27353,349891_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27354,144723_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27355,163905_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
dataset1g = dataset1g.applymap(lambda x: np.log1p(x) if isinstance(x, (int, float)) else x)
mean_values = dataset1g.mean()
dataset1g.fillna(mean_values, inplace = True)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1g)
X, y = process_dataset(dataset2, user_features_gen)

Processing rows: 100%|██████████| 25706/25706 [00:25<00:00, 1007.48it/s]


In [26]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, precision_score, recall_score

scaler = MinMaxScaler()
combined2 = scaler.fit_transform(X)
columns = [f"Feature_{i}" for i in range(combined2.shape[1])]
combined2_df_s = pd.DataFrame(combined2, columns=columns)
X = combined2_df_s

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

precision_test = precision_score(y_test, y_pred)
recall_test = recall_score(y_test, y_pred)
print("Precision for Test Set:", precision_test)
print("Recall for Test Set:", recall_test)

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

precision_val = precision_score(y_validate, y_pred_validate)
recall_val = recall_score(y_validate, y_pred_validate)
print("Precision for Validation Set:", precision_val)
print("Recall for Validation Set:", recall_val)

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Accuracy: 0.6738250855897915
Confusion Matrix:
[[1188  409]
 [ 639  977]]
Precision for Test Set: 0.7049062049062049
Recall for Test Set: 0.6045792079207921
FPR for Test Set: 0.3512836568566061
TPR for Test Set: 0.6497524752475248
EER for Test Set: 0.3507655908045406
Validation Accuracy: 0.6776602364654636
Validation Confusion Matrix:
[[1236  374]
 [ 662  942]]
Precision for Validation Set: 0.71580547112462
Recall for Validation Set: 0.587281795511222
FPR for Validation Set: 0.3515527950310559
TPR for Validation Set: 0.6502493765586035
EER for Validation Set: 0.35065170923622624
